In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np


In [2]:
# 1. Hyper Parameter
train_size = 784
num_classes = 10
hidden1_size = 500
learning_rate = 0.001

batch_size = 100
ephoc_size = 5


In [3]:
# 2. Data load
# MNIST Dataset 
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

0it [00:00, ?it/s]

100%|█████████▉| 9904128/9912422 [01:06<00:00, 140190.63it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz



0it [00:00, ?it/s]


 57%|█████▋    | 16384/28881 [00:00<00:00, 45124.80it/s]
32768it [00:00, 38003.34it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz



 70%|██████▉   | 1146880/1648877 [00:11<00:05, 95467.62it/s] 
9920512it [01:20, 140190.63it/s]                             [A
 99%|█████████▉| 1630208/1648877 [00:16<00:00, 163567.71it/s]

0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz




  0%|          | 0/4542 [00:01<?, ?it/s]

8192it [00:01, 5339.75it/s]             

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [4]:
# 3. Build the Model
class FeedForwardNN(nn.Module):
    def __init__(self, train_size, hidden1_size, num_classes):
        super(FeedForwardNN, self).__init__()
        self.linear1 = nn.Linear(train_size, hidden1_size) #784x20
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden1_size, num_classes) #20x10
        
    def forward(self, x):
        z1 = self.linear1(x)
        a1 = self.relu(z1)
        z2 = self.linear2(a1)
        
        return z2
        

In [5]:
# 4. Generate a model
model = FeedForwardNN(train_size, hidden1_size, num_classes)

In [6]:
# 5. Init loss function and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# 6. Train
for ephoc in range(ephoc_size):
    print("ephoc: ",ephoc)
    for idx, (images, labels) in enumerate(train_loader):
        #convert dataset as the Pytorch style
        images = Variable(images.view(-1,28*28))
        labels = Variable(labels)
        
        #Forward, Backward, gradient
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if idx%100 == 0:
            print("ephoc[",ephoc,"] \t", "loss:", loss.item())
            

ephoc:  0
ephoc[ 0 ] 	 loss: 2.2076711654663086
ephoc[ 0 ] 	 loss: 0.23751743137836456
ephoc[ 0 ] 	 loss: 0.24504905939102173
ephoc[ 0 ] 	 loss: 0.2440721094608307
ephoc[ 0 ] 	 loss: 0.14754512906074524
ephoc[ 0 ] 	 loss: 0.12520276010036469
ephoc:  1
ephoc[ 1 ] 	 loss: 0.16524162888526917
ephoc[ 1 ] 	 loss: 0.10681936144828796
ephoc[ 1 ] 	 loss: 0.055490948259830475
ephoc[ 1 ] 	 loss: 0.11570068448781967
ephoc[ 1 ] 	 loss: 0.07411623001098633
ephoc[ 1 ] 	 loss: 0.052554111927747726
ephoc:  2
ephoc[ 2 ] 	 loss: 0.08383353054523468
ephoc[ 2 ] 	 loss: 0.03309843689203262
ephoc[ 2 ] 	 loss: 0.04478994011878967
ephoc[ 2 ] 	 loss: 0.08570170402526855
ephoc[ 2 ] 	 loss: 0.014440944418311119
ephoc[ 2 ] 	 loss: 0.02342766337096691
ephoc:  3
ephoc[ 3 ] 	 loss: 0.011843901127576828


KeyboardInterrupt: 

In [9]:
#Test the model
total = 0
correct = 0
for images, labels in test_loader:
    images = Variable(images.view(-1,28*28))

    outputs = model(images)   
    _, predicted = torch.max(outputs.data, 1)
    total += len(predicted)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %
